In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

Collecting enterprise-specific data: flex-CSS

In [1]:
DEBUG = 1

def connectchrome():
    options = Options()
    # if you want to hide browser window, uncomment line below
    #options.add_argument('--headless')
    options.add_argument('log-level=3')
    options.add_argument("--incognito")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    driver = webdriver.Chrome(executable_path= r'C:\\Users\\user\\Desktop\\python\\banyous\\1-Quora-scrapping\\chromedriver.exe', options = options)
    driver.maximize_window()
    time.sleep(2)
    return driver

driver = connectchrome()
driver.implicitly_wait(3)

# Log-in
driver.get('https://www.businessmodelideas.com/login')
login_button = driver.find_element_by_xpath("//div[@class='col-12 text-lg-right text-left']/button[@type='submit' and @class='cta']")
driver.find_element_by_name('login').send_keys('innolab.admin@ds.seoultech.ac.kr') #user_id
driver.find_element_by_name('password').send_keys('toctoctoc') #password
login_button.click()
driver.implicitly_wait(3)

# Read a text file with business names
with open('company_list(N=156).txt', 'r') as file:
    company_names = [line.strip() for line in file.readlines()]
    #company_names = [line.strip().lower() for line in file.readlines()]
    
all_data = []

for company_name in company_names: 
    try:
        url = f"https://www.businessmodelideas.com/company/{company_name}"
        driver.get(url)

        data = {"Company": company_name}
        description_element = driver.find_element_by_xpath("//div[@class='company-description']/div[@class='pr-lg-5 normal-text mb-4']/p")
        description_text = description_element.text
        data["Description"] = description_text

        headers = ["Key Partners", "Key Activities", "Key Resources", "Value Propositions", 
                "Customer Relationships", "Channels", "Customer Segments", 
                "Cost Structure", "Revenue Streams"]

        for header in headers:
            header_elements = driver.find_elements_by_xpath(f"//div[@class='cube-heading mb-2'][contains(text(), '{header}')]/following-sibling::div[contains(@class, 'cube-element')]/span")
            header_texts = ', '.join([element.text for element in header_elements])
            data[header] = header_texts

        all_data.append(data)
    except NoSuchElementException as e:
        print(f"Cannot find data for {company_name}. Skipping this company.")

# Convert all enterprise data into a single DataFrame
df_all = pd.DataFrame([all_data])

# Save the DataFrame as a CSV file
df_all.to_csv("collected_data_5.csv", index=False)
    
# Print the number of companies that were successfully collected
print(f"Number of companies successfully scraped: {len(all_data)}")

# Close the Webdriver
driver.quit()

NameError: name 'Options' is not defined